In [1]:
from IPython.display import display, Markdown

In [2]:
# do bm25 first then vector? nope. maybe just bm25

In [3]:
from sub_agents import knowledge_agent
# knowledge agent can answer questions from  multiple databases
# when the model gets smarter - i only need retrieval, no rag chain needed within the knowledge function

/Users/arjun/Documents/github/smart-agent/gemini-agent/data/ec_sop.csv
csv present


In [4]:
query = "what is internal settlement?"

In [5]:
answer = knowledge_agent.execute_task(query)

An internal settlement transaction is a transaction between two Euroclear Bank clients. The transaction settles in Euroclear Bank and can be either:

* against payment in any of the payment currencies eligible in Euroclear Bank provided that the security has the multi-currency treatment flag open
* free of payment


In [6]:
Markdown(answer)

Internal settlement is a transaction between two Euroclear Bank clients. The transaction settles in Euroclear Bank and can be either:
* against payment in any of the payment currencies eligible in Euroclear Bank provided that the security has the multi-currency treatment flag open 
* free of payment